# Deploy Model to online endpoint

- Models created with MLFlow do not require a scoring script nor an environment, with this notebook, we will create a scopy.py file and use that with our model.

- Before running this notebook, run the **Chapter 6 Prep-Model Creation & Registration.ipynb** notebook to create and register a model for use

- Models created with MLFlow do not require a scoring script nor an environment

## In this notebook we will:

- Connect to your workspace.
- Create an online endpoint
- Retrieve and register a model from the job ran in the previous notebook
- Create a deployment
- Make an API Call to the managed online endpoint

Let's get started

You can use either the Python 3.10 - SDK V2 kernel, or your job_env kenel to run this notebook.

**Kernel** > **Change Kernel** > **Python 3.10 - SDK V2**

or if you already setup the virtual environment in Chapter 4:

Select **Kernel** > **Change Kernel** > **job_env**


In [1]:
import azure.ai.ml
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

print(azure.ai.ml._version.VERSION)

1.1.2


In [2]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [3]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

### Configure endpoint

In [4]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "ch6-sdkv2-endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="titanic online endpoint for mlflow model",
    auth_mode="key",
    tags={"oneline endpoint": "titanic"},
)

## Create endpoint

Using the MLClient created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.


In [5]:
ml_client.begin_create_or_update(endpoint)

In [6]:
import time
print(online_endpoint_name)
while ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state == 'Creating':
    print('Creating')
    time.sleep(15)

print(ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state)

ch6-sdkv2-endpt-12102241150428
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Succeeded


## Create deployment
A deployment is a set of resouces used for hosting the inferecing model using the *ManagedOnlineDeployment* class.  
Using the *ManagedOnlineDeployment* class, a developer can configure the following components

- name: name of the deployment
- endpoint_name: name of the endpoint to create the deployment under
- model: the model to use for the deployment
- instance_type: the VM side to use for deployment
- instance_count: the number of instances to use for the deployment
    

# Retrieve Model from registry

In [7]:
import mlflow
import os
import shutil

In [8]:
experiment = 'chapter6'
current_experiment=dict(mlflow.get_experiment_by_name(experiment))
experiment_id=current_experiment['experiment_id']
print(experiment_id)

df = mlflow.search_runs([experiment_id])
run_id = df['run_id'].iloc[-1]
print(run_id)
print(type(run_id))

mlflow.set_experiment(experiment_name='chapter6')
client = mlflow.tracking.MlflowClient()
client.list_artifacts(run_id=run_id)

file_path = client.download_artifacts(
    run_id, path="model"
)
shutil.copytree(file_path, './model', dirs_exist_ok=True)

model = Model(path="./model/model.pkl")

7b6a371b-4dc8-435d-8358-e482be62200e
tidy_cassava_0ll0gyn6mv
<class 'str'>


/tmp/ipykernel_596917/3874377315.py:15: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  file_path = client.download_artifacts(


## Create Deployment Environment

- MLFlow Models by default create a conda.yaml file which is used with an MLflow base image which contains:
    - azureml-inference-server-http
    - mlflow-skinny
    
- In this example, we are downloading the model, so we need to specify the packages we need installed since we are not using the no-code deployment.  

In [9]:
%%writefile conda-yamls/env_for_sdkv2deploy.yml
name: job_env_for_build
dependencies:
- python=3.10
- scikit-learn=1.1.3
- ipykernel
- matplotlib
- pandas
- pip
- pip:
  - azureml-defaults==1.48.0 #needed for the inferece schema
  - mlflow<=1.30.0
  - azure-ai-ml==1.1.2
  - mltable==1.0.0
  - azureml-mlflow==1.48.0

Overwriting conda-yamls/env_for_sdkv2deploy.yml


In [10]:
from azure.ai.ml.entities import Environment, BuildContext

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    conda_file="conda-yamls/env_for_sdkv2deploy.yml",
    name="env_for_sdkv2deploy",
    description="Environment created from a Docker image plus Conda environment.",
)
env = ml_client.environments.create_or_update(env_docker_conda)

In [11]:
env

Environment({'is_anonymous': False, 'auto_increment_version': False, 'name': 'env_for_sdkv2deploy', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {}, 'id': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourceGroups/chapter6-azureml-rg/providers/Microsoft.MachineLearningServices/workspaces/chapter6-azureml/environments/env_for_sdkv2deploy/versions/5', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/memasanz2/code/Users/memasanz/ML-Engineering-with-Azure-Machine-Learning-Service/chapter 6', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7fd4888b0610>, 'serialize': <msrest.serialization.Serializer object at 0x7fd4888b0d90>, 'version': '5', 'latest_version': None, 'conda_file': {'dependencies': ['python=3.10', 'scikit-learn=1.1.3', 'ipykernel', 'matplotlib', 'pandas', 'pip', {'pip': ['azureml-defaults==1.48.0', 'mlflow<=1.30.0', 'azure-ai-ml==1.

## Creating Scoring Script

score.py file

In [12]:
import os

# Create a folder for the experiment files
script_folder = 'ManagedOnlineEndpoint'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

ManagedOnlineEndpoint folder created


In [13]:
%%writefile $script_folder/score.py

import os 
import json
import joblib
from pandas import json_normalize
import pandas as pd
import logging

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
    model = joblib.load(model_path)
    logging.info("Init complete")

# Called when a request is received
def run(raw_data):
    dict= json.loads(raw_data)
    df = json_normalize(dict['raw_data']) 
    y_pred = model.predict(df)
    print(type(y_pred))
    
    result = {"result": y_pred.tolist()}
    return result

Overwriting ManagedOnlineEndpoint/score.py


## Configure the deployment

In [14]:
import datetime


blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="./ManagedOnlineEndpoint", scoring_script="score.py"
    ),
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

## Create deployment

In [15]:
ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint ch6-sdkv2-endpt-12102241150428 exists
Uploading ManagedOnlineEndpoint (0.0 MBs): 100%|██████████| 940/940 [00:00<00:00, 31502.61it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


In [16]:
import time
while ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state == 'Updating':
    print('Updating..will take about 10 minutes to deploy...')
    time.sleep(15)
    
ml_client.online_deployments.get(name = "blue", endpoint_name = online_endpoint_name).provisioning_state

Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy...
...Updating..will take about 10 minutes to deploy

'Succeeded'

In [17]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint)

In [18]:
import time
while ml_client.online_endpoints.get(name=online_endpoint_name).provisioning_state == 'Updating':
    print('Updating')
    time.sleep(15)
    
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
endpoint.provisioning_state

Updating
Updating


'Succeeded'

In [19]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(endpoint)
print(' ')
# existing traffic details
print(endpoint.traffic)
print(' ')
# Get the scoring URI
print('uri: ' + str(endpoint.scoring_uri))
primary_key = ml_client.online_endpoints.get_keys(name = online_endpoint_name).primary_key
print(' ')
print('primary key: ' + str(primary_key))

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://ch6-sdkv2-endpt-12102241150428.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://ch6-sdkv2-endpt-12102241150428.eastus.inference.ml.azure.com/swagger.json', 'name': 'ch6-sdkv2-endpt-12102241150428', 'description': 'titanic online endpoint for mlflow model', 'tags': {'oneline endpoint': 'titanic'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/resourcegroups/chapter6-azureml-rg/providers/microsoft.machinelearningservices/workspaces/chapter6-azureml/onlineendpoints/ch6-sdkv2-endpt-12102241150428', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/5da07161-3770-4a4b-aa43-418cbbb627cf/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:9e44248b-9077-4922-99ad-0c577c53684e:6b0c72f7-7c63-45b3-8456-25c058b85b93?api-version=2022-02-01-preview'}, 'id': '/subscription

In [20]:
import pandas as pd
df = pd.read_csv('./prepped_data/titanic_prepped.csv')
columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep].head(5)
X_raw

,Embarked,Loc,Sex,Pclass,Age,Fare,GroupSize
0,S,X,m,3,22.0,7.2500,2
1,C,C,f,1,38.0,71.2833,2
2,S,X,f,3,26.0,7.9250,1
3,S,C,f,1,35.0,53.1000,2
4,S,X,m,3,35.0,8.0500,1


In [21]:
import json
url = endpoint.scoring_uri
api_key = primary_key  # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
import requests

def make_prediction(df):
    endpoint_url = url
    body = df.to_json(orient='records') 
    body = '{"raw_data": ' + body + '}'
    print(body)
    r = requests.post(endpoint_url, headers=headers, data=body)
    return (r.json())


columns_to_keep =  ['Embarked', 'Loc', 'Sex','Pclass', 'Age', 'Fare', 'GroupSize']
X_raw           = df[columns_to_keep]


dftest = X_raw.head(5)

results = make_prediction(dftest)

print(results)
val = results['result']
print('')
print('predictions')
print(val)

{"raw_data": [{"Embarked":"S","Loc":"X","Sex":"m","Pclass":3,"Age":22.0,"Fare":7.25,"GroupSize":2},{"Embarked":"C","Loc":"C","Sex":"f","Pclass":1,"Age":38.0,"Fare":71.2833,"GroupSize":2},{"Embarked":"S","Loc":"X","Sex":"f","Pclass":3,"Age":26.0,"Fare":7.925,"GroupSize":1},{"Embarked":"S","Loc":"C","Sex":"f","Pclass":1,"Age":35.0,"Fare":53.1,"GroupSize":2},{"Embarked":"S","Loc":"X","Sex":"m","Pclass":3,"Age":35.0,"Fare":8.05,"GroupSize":1}]}
{'result': [0, 1, 0, 1, 0]}

predictions
[0, 1, 0, 1, 0]
